# 사방사업현황 vs 산사태발생

In [ ]:
사방사업현황 -> 경기도 산사태위험지역을 대상으로 해마다 진행되는 사방사업 현황입니다.

가설 : 태양광발전소, 터널 및 도로 등 건설은 산사태에 영향을 줄 것이다.
-> 산사태 많은 지역 혹은 위험지역과 사방사업현황을 비교
-> 적절한 지역에 사방사업이 진행되고 있는지를 분석
-> 시군구명을 기준으로 합계사업비와 산사태발생횟수를 비교

In [29]:
# 경고메세지 제거
import warnings
warnings.filterwarnings('ignore')

# 모듈 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc

# 한글처리
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

# 사방사업현황

## 데이터셋 로드

In [56]:
df = pd.read_csv('사방사업현황.csv', encoding ='euc-kr')

## 데이터 특성 확인

In [57]:
df.head(1)

,사업완료년도,관리번호,시행상위기관명,시행하위기관명,시공소재지시도명,시공소재지시군구명,시공소재지읍면동명,시공소재지리명,국비(백만원),지방비(백만원),합계사업비(백만원)
0,2018,경기-00838,경기도,산림환경연구소,경기도,김포,통진,서암,50.0,22.0,72.0


In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   사업완료년도      900 non-null    int64  
 1   관리번호        883 non-null    object 
 2   시행상위기관명     900 non-null    object 
 3   시행하위기관명     900 non-null    object 
 4   시공소재지시도명    900 non-null    object 
 5   시공소재지시군구명   900 non-null    object 
 6   시공소재지읍면동명   830 non-null    object 
 7   시공소재지리명     857 non-null    object 
 8   국비(백만원)     571 non-null    float64
 9   지방비(백만원)    571 non-null    float64
 10  합계사업비(백만원)  680 non-null    float64
dtypes: float64(3), int64(1), object(7)
memory usage: 77.5+ KB


In [59]:
df.describe()

,사업완료년도,국비(백만원),지방비(백만원),합계사업비(백만원)
count,900.000000,571.000000,571.000000,680.000000
mean,2010.542222,120.483187,50.509632,157.758824
std,6.956515,69.677010,31.157402,96.295985
min,1986.000000,10.000000,0.000000,6.000000
25%,2007.000000,72.500000,29.000000,96.000000
50%,2013.000000,103.000000,42.000000,136.000000
75%,2015.000000,158.000000,65.500000,200.000000
max,2018.000000,675.000000,289.000000,964.000000


In [60]:
df.shape

(900, 11)

In [61]:
df.isnull().sum()

사업완료년도          0
관리번호           17
시행상위기관명         0
시행하위기관명         0
시공소재지시도명        0
시공소재지시군구명       0
시공소재지읍면동명      70
시공소재지리명        43
국비(백만원)       329
지방비(백만원)      329
합계사업비(백만원)    220
dtype: int64

In [62]:
df['사업완료년도'].value_counts()

2012    120
2014    118
2013    100
2016     66
2017     66
2018     61
2015     61
2007     38
2010     27
2005     27
2006     23
2011     22
2009     20
1999     16
2001     14
2008     13
2004     12
2003     12
2002     11
1997      9
1990      8
1998      7
1989      7
1995      6
1986      6
1991      5
1988      5
2000      4
1996      4
1993      4
1994      3
1987      3
1992      2
Name: 사업완료년도, dtype: int64

In [63]:
max(df['사업완료년도'])

2018

## 데이터 전처리

### 사업완료년도 2013~2018년도로 추출

In [64]:
df_copy = df.copy()

In [65]:
df = df.loc[df['사업완료년도'] >= 2013]

In [66]:
df['사업완료년도'].value_counts()

2014    118
2013    100
2017     66
2016     66
2018     61
2015     61
Name: 사업완료년도, dtype: int64

### 결측치 처리 -> 합계사업비 널값에 국비+지방비으로 널값처리

In [67]:
# 국비(백만원), 지방비(백만원) 널값 제거
df = df.dropna(subset=['국비(백만원)', '지방비(백만원)'])


# 합계 널값인 경우)
# 1. 국비 지방비 존재o -> 합계 X -> 국비+지방비 합계로 대체
# 2. 국비 지방비 존재x -> 합계 X -> 합계 0 => 사업 진행x (drop)

# 발생지역 a) 산사태o + 사업 진행x -> merge에서 outer로 하면 확인 가능

In [68]:
df.isnull().sum()

사업완료년도         0
관리번호           0
시행상위기관명        0
시행하위기관명        0
시공소재지시도명       0
시공소재지시군구명      0
시공소재지읍면동명     39
시공소재지리명       19
국비(백만원)        0
지방비(백만원)       0
합계사업비(백만원)    99
dtype: int64

In [69]:
df['합계사업비(백만원)'] = df['국비(백만원)'] + df['지방비(백만원)']

In [70]:
df.isnull().sum()

사업완료년도         0
관리번호           0
시행상위기관명        0
시행하위기관명        0
시공소재지시도명       0
시공소재지시군구명      0
시공소재지읍면동명     39
시공소재지리명       19
국비(백만원)        0
지방비(백만원)       0
합계사업비(백만원)     0
dtype: int64

### 불필요한 컬럼 제거

In [71]:
df.drop(['사업완료년도', '관리번호', '시행상위기관명', '시행하위기관명', '시공소재지시도명', '시공소재지읍면동명', '시공소재지리명', '국비(백만원)', '지방비(백만원)'], axis=1, inplace=True)

In [72]:
df.head(1)

,시공소재지시군구명,합계사업비(백만원)
0,김포,72.0


In [73]:
df.isnull().sum()

시공소재지시군구명     0
합계사업비(백만원)    0
dtype: int64

In [74]:
df.shape

(471, 2)

In [75]:
df.rename(columns={'시공소재지시군구명':'지역명'},inplace=True)

In [76]:
#df = df.reset_index().rename(columns={"index":"지역명"})

In [78]:
df.head()

,지역명,합계사업비(백만원)
0,김포,72.0
1,남양주,79.0
2,김포,134.0
3,군포,367.0
4,광주,230.0


# 산사태 현황

## 데이터셋 로드

In [79]:
data = pd.read_csv('산사태_현황.csv')

## 데이터 특성 확인

In [80]:
data.head(1)

,시도명,읍면명,산사태피해면적,산사태재해명,산사태재해년도,시군구명
0,경기도,단월면,3.15,7.28.~8.11.,2020,양평군


In [81]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   시도명      248 non-null    object 
 1   읍면명      248 non-null    object 
 2   산사태피해면적  248 non-null    float64
 3   산사태재해명   248 non-null    object 
 4   산사태재해년도  248 non-null    int64  
 5   시군구명     248 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 11.8+ KB


In [82]:
data.describe()

,산사태피해면적,산사태재해년도
count,248.000000,248.000000
mean,0.675202,2019.818548
std,0.655024,0.572072
min,0.010000,2018.000000
25%,0.255000,2020.000000
50%,0.500000,2020.000000
75%,0.805000,2020.000000
max,3.430000,2020.000000


In [83]:
data.shape

(248, 6)

In [85]:
data['산사태재해년도'].value_counts()

2020    225
2018     22
2019      1
Name: 산사태재해년도, dtype: int64

## 불필요한 칼럼 제거

In [86]:
# 원본 복사해두기
data_copy = data.copy()

In [87]:
data.drop(['시도명', '읍면명', '산사태재해명', '산사태재해년도'], axis=1, inplace=True)

In [88]:
data.head()

,산사태피해면적,시군구명
0,3.15,양평군
1,2.57,양평군
2,2.28,양평군
3,2.19,양평군
4,1.96,이천시


In [89]:
data.rename(columns={'시군구명':'지역명'},inplace=True)

In [91]:
data = data['지역명'].value_counts().to_frame()

In [92]:
data.rename(columns={"지역명":'산사태발생횟수'},inplace=True)

In [93]:
data = data.reset_index().rename(columns={"index":"지역명"})

In [94]:
data

,지역명,산사태발생횟수
0,안성시,94
1,이천시,45
2,연천군,24
3,용인시 처인구,24
4,가평군,19
5,양평군,16
6,양주시,10
7,포천시,5
8,김포시,3
9,파주시,2


In [95]:
tp = pd.merge(df, data, how='outer', on='지역명')

In [96]:
tp

,지역명,합계사업비(백만원),산사태발생횟수
0,김포,72.0,NaN
1,김포,134.0,NaN
2,김포,136.0,NaN
3,김포,271.0,NaN
4,김포,271.0,NaN
...,...,...,...
474,김포시,NaN,3.0
475,여주시,NaN,2.0
476,용인시 수지구,NaN,1.0
477,평택시,NaN,1.0


In [99]:
tp.to_csv('result.csv', index=False, )

In [98]:
import csv

f = open('result.csv', 'w', newline='')
wr = csv.writer(f)
wr.writerrow(tp['지역명'])

f.close()

AttributeError: '_csv.writer' object has no attribute 'writerrow'